In [2]:
#Importing the libraries
import numpy as np
from tqdm import tqdm
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'LICENSE',
 'README.md',
 'scifi-stories-text-corpus',
 'scifi-stories-text-corpus.zip',
 'Text Generation.ipynb']

In [4]:
os.listdir("scifi-stories-text-corpus/")

['internet_archive_scifi_v3.txt']

In [ ]:
!sudo chmod 777 internet_archive_scifi_v3.txt
!stat -c "%a %n" internet_archive_scifi_v3.txt

In [4]:
#Load the raw text and convert them to lowercase
filename = "scifi-stories-text-corpus/internet_archive_scifi_v3.txt"
text_corpus = open(filename).read()
text_corpus = text_corpus.lower()

In [5]:
#Create mapping from characters to integers
chars = sorted(list(set(text_corpus)))
chars_to_int = dict((char, integer) for integer, char in enumerate(chars))

In [9]:
#Print the unique characters present in the corpus
print(chars_to_int.keys())

dict_keys([' ', '!', '"', '#', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])


In [6]:
#Basic information about the dataset
n_chars = len(text_corpus)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  149326361
Total Vocab:  49


In [ ]:
#Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
X_train = []
y_train = []
for i in tqdm(range(0, n_chars - seq_length, 1)):
    input_sequence = text_corpus[i:i + seq_length]
    output_sequence = text_corpus[i + seq_length]
    X_train.append([chars_to_int[char] for char in input_sequence])
    y_train.append(chars_to_int[output_sequence])
n_patterns = len(X_train)
print ("Total Patterns: ", n_patterns)